In [46]:
import flywheel
import pandas as pd
import datetime

fw = flywheel.Client()
collection_id = "60eca24fcfdec364fd96c874"
collection = fw.get(collection_id)
gear = 'wscore-ct-heatmap'
new_collection = fw.get("618d6fb6bb845b310730d8f8")

In [49]:
collection_sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [55]:
# Create a data dict:
data_dict = {'subject':[],'session':[], 'project':[],'run':[],'status':[], 'label': []}

# Iterate over sessions
for session in collection_sessions:

    # Because we want information off the sessions's analyses, we need to reload
    # The container to make sure we have all the metadata.
    session = session.reload()
    sub_label = session.subject.label
    ses_label = session.label
    project = fw.get(session.parents['project']).label
    # Any analyses on this session will be stored as a list:
    analyses = session.analyses

    # If there are no analyses containers, we know that this gear was not run
    if len(analyses) == 0:
        run = 'False'
        status = 'NA'
        label = 'NA'
    else:
        # Loop through the analyses
        matches = [asys for asys in analyses if asys.gear_info and gear in asys.gear_info.get('name')]
        # If there are no matches, the gear didn't run
        if len(matches) == 0:
            run = 'False'
            status = 'NA'
            label = 'NA'
        # If there is one match, that's our target
        elif len(matches) == 1:
            run = 'True'
            status = matches[0].job.get('state')
            label = matches[0].label
        # If there are more than one matches (due to reruns), take the most recent run.
        # This behavior may be modified to whatever suits your needs
        else:
            last_run_date = max([asys.created for asys in matches])
            last_run_analysis = [asys for asys in matches if asys.created == last_run_date]

            # There should only be one exact match
            last_run_analysis = last_run_analysis[0]

            run = 'True'
            status = last_run_analysis.job.get('state')
            label = last_run_analysis.label

    # Populate our data dict - remember that each key in the data dict must be updated
    # So that the length of our lists stays the same
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['run'].append(run)
    data_dict['status'].append(status)
    data_dict['label'].append(label)
    data_dict['project'].append(project)

# Now create a data frame
df = pd.DataFrame.from_dict(data_dict)
df

,subject,session,project,run,status,label
0,123194,123194x20190603x3T,NACC-SC,True,complete,wscore_2021-11-8_15:57_WT_PVS_sx
1,119851,119851x20190613x3T,NACC-SC,True,complete,wscore_2021-11-8_15:57_WT_PVS_sx
2,121105,121105x20170427x3T,NACC-SC,True,complete,wscore_2021-07-12_WT
3,121195,121195x20170523x3T,NACC-SC,True,failed,wscore_2021-07-19_WT
4,121299,121299x20170808x3T,NACC-SC,True,complete,wscore_2021-07-12_WT
5,122005,122005x20171213x3T,NACC-SC,True,complete,wscore_2021-07-12_WT
6,101453,101453x20181016x3T,NACC-SC,True,complete,wscore_2021-11-8_15:57_WT_PVS_sx
7,120131,120131x20180315x3T,NACC-SC,True,complete,wscore_2021-07-12_WT
8,120515,120515x20181127x3T,NACC-SC,True,complete,wscore_2021-11-8_15:57_WT_PVS_sx
9,120965,120965x20180821x3T,NACC-SC,True,complete,wscore_2021-11-8_15:57_WT_PVS_sx


In [56]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
x = datetime.datetime.now()
time_str = '%s_%s_%s-%s_%s_%s' % (x.year, x.month, x.day, x.hour, x.minute, x.second) # for the report filename 

csv_out = f'../../{gear}_RunReport_OldCollection_{time_str}.csv'

df.to_csv(csv_out,index=False)

#new_collection.upload_file(csv_out)